In [ ]:
from phi3v import FinetunePhi3V
from qwenvl import FinetuneQwenVL

# Sample data
data = [
    {
        "image": "image.jpg",
        "input": "Describe the image.",
        "output": "This is a picture of two bags."
    },
    {
        "image": "image.jpg",
        "input": "Describe the image.",
        "output": "This is a picture of two bags."
    },
        {
        "image": "image.jpg",
        "input": "Describe the image.",
        "output": "This is a picture of two bags."
    },
            {
        "image": "image.jpg",
        "input": "Describe the image.",
        "output": "This is a picture of two bags."
    },
    # Add more data points as needed
]

# Initialize the finetune class with the data
finetuner = FinetuneQwenVL(
    data=data,
    epochs=300,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    optim="adamw_hf",
    model_id="unsloth/Qwen2-VL-7B-Instruct",
    peft_r=16,
    peft_alpha=32,
    peft_dropout=0.1,
)

# Run the finetuning process
finetuner.run()


==((====))==  Unsloth 2025.1.6: Fast Qwen2_Vl vision patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.684 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from unsloth import FastVisionModel
from PIL import Image

model_id = "unsloth/Qwen2-VL-7B-Instruct"
model, tokenizer = FastVisionModel.from_pretrained(
    model_name =  model_id,  # Trained model either locally or from huggingface
    load_in_4bit = False,
)

image = Image.open("image.jpg")
instruction = "Describe the image."

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {
                "type": "text",
                "text": instruction
            },
        ]
    }
]



FastVisionModel.for_inference(model)         # Enable for inference!
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")
# text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# output = model.generate(**inputs, max_new_tokens = 1024,
#                    use_cache = True, temperature = 1.5, min_p = 0.1)
output_ids = model.generate(
    **inputs,
    max_new_tokens=1024,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

# Decode generated tokens into text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

# Print the generated text
print("Generated Output:", generated_text)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/cqilab/anaconda3/envs/llmfinetune/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.6: Fast Qwen2_Vl vision patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.684 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


Generated Output: system
You are a helpful assistant.
user
Describe the image.
assistant
The image shows two black backpacks placed side by side on a surface. The backpack on the left is larger and has a longer shoulder strap, while the backpack on the right is smaller and has a shorter shoulder strap. Both backpacks have zippers and seem to be made of a similar material. The background appears to be a light-colored surface, possibly a table or a counter. The text "Let's Go Digital" is visible in the bottom right corner of the image.


In [ ]:
    generate_ids = output[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]